In [24]:
## remember to run conn_cocolab from the terminal before running cells in this notebook!
import os
import urllib, cStringIO

import pymongo as pm

import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

import numpy as np
from __future__ import division
import scipy.stats as stats
import pandas as pd
import json
import re

from PIL import Image
import base64
import datetime

# import svg_render_helpers as rdh
from scipy.spatial import distance

### file hierarchy and database connection vars

In [25]:
## directory & file hierarchy
# iterationName = 'CDM_photodraw_e2_testing'
iterationName = 'CDM_photodraw_e2'
exp_path = 'photodraw'
analysis_dir = os.getcwd()
exp_dir = os.path.abspath(os.path.join(os.getcwd(),'../..','experiments'))
##
sketch_dir = os.path.join(analysis_dir,'sketches_CDM_included_only')
if not os.path.exists(sketch_dir):
    os.makedirs(sketch_dir)

## dir where we can keep all sketches not binned by category    

In [26]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['kiddraw']
coll = db['CDM_photodraw_e2']


### Preprocessing data and rendering images

Notes:
There are two types of records in the database: 'stroke' and 'finalImage'. This is stored under the key: 'dataType'.
The 'stroke'-type of record contains the svg string information. Every stroke event is stored as a separate record.
The session identifier is called "sessionId".




In [27]:
total_sessions = coll.find().distinct('sessionId') ## this returns ALL sessions in this collection.
print 'We currently have {} total sessions.'.format(len(total_sessions))

We currently have 0 total sessions.


In [21]:
really_run_this = 1
if really_run_this:
    
    ## img writing parameters
    imsize = 224
    
    ## desired output: a dataframe that has trials on the rows, and the following columns:
    ## category, age, number of strokes, mean_pixel_intensity, bounding_box_coordinates_LTRB, list of strokes, 
    ## PNG string, submission_time, submission_date
    ## to be saved out as a nice tidy CSV
    session_id = []
    trial_num = []
    category = []
    num_strokes = []
    mean_pixel_intensity = []
    bounding_box_coords = []
    svg = []
    svg_times = []
    png = []
    submit_time = []
    submit_date = []
    draw_duration = []
    draw_duration_strokes = []
    filename = []
    age=[]
    countErr = 0
    condition=[]
    readadble_date = []
    exclude_session_id = []
    include_age = ['age4']

    for s in total_sessions:
        this_collection = coll
        image_recs = this_collection.find({'$and': [{'sessionId':s}, {'dataType':'finalImage'}]}).sort('time')  
        
        for imrec in image_recs:  
                if imrec['sessionId'] not in exclude_session_id and imrec['age'] in include_age:
                    stroke_recs = this_collection.find({'$and': [ 
                                                      {'sessionId':s}, 
                                                      {'dataType':'stroke'},
                                                      {'category': imrec['category']}]}).sort('time')   

                    if stroke_recs.count() > 0: ## only include trials if the drawings are not blank            
                        session_id.append(imrec['sessionId'])        
                        trial_num.append(imrec['trialNum']) 
                        category.append(imrec['category'])

                        try:    
                            png.append(imrec['imgData'])
                            submit_time.append(imrec['endTrialTime'])
                            submit_date.append(imrec['date'])
                            age.append(imrec['age'])
                            condition.append(imrec['condition'])
                            filename.append(os.path.join(sketch_dir,'{}_sketch_{}_{}_{}.png'.format(imrec['category'], imrec['sessionId'], imrec['condition'], imrec['age'])))
                            num_strokes.append(stroke_recs.count())
                            draw_duration.append(imrec['endTrialTime'] - imrec['startTrialTime'])

                            _svg = [] # this keeps track of the strokes from THIS final image
                            _svg_times = []
                            for strec in stroke_recs:
                                _svg.append(strec['svg'])
                                _svg_times.append(strec['endStrokeTime'])
                            draw_duration_strokes.append((_svg_times[-1] - _svg_times[0])/1000) ## in seconds

                            ## and write out image data here too
                            imgData = imrec['imgData'];
                            filestr = base64.b64decode(imgData)
                            child_dir = os.path.join(sketch_dir,imrec['sessionId'])
                            if not os.path.exists(child_dir):
                                os.makedirs(child_dir)
                            fname = os.path.join(child_dir,'{}_sketch_{}_{}_{}.png'.format(imrec['category'],imrec['sessionId'],imrec['condition'], imrec['age']))
                            with open(fname, "wb") as fh:
                                fh.write(imgData.decode('base64'))  
                            
                        except Exception as e:
                            print('something went wrong with an image')
                            print e
                            countErr = countErr + 1
                            print imrec['category']
                            child_dir = os.path.join(sketch_dir,'noname')
                            if not os.path.exists(child_dir):
                                os.makedirs(child_dir)
                            fname = os.path.join(child_dir,'{}_sketch_{}_{}.png'.format(imrec['category'], 'noname',imrec['sessionId']))
                            with open(fname, "wb") as fh:
                                fh.write(imgData.decode('base64'))                   

In [ ]:
## How many session IDs do we have?
count_subs = np.size(np.unique(session_id))
print 'We have {}  subjects after exclusions.'.format(count_subs)

In [22]:
## How many drawings do we have in each condition?  
print 'We have {} semantic drawings.'.format(condition.count("S"))
print 'We have {} perception drawings.'.format(condition.count("P")

We have 0 wm drawings.
We have 12 semantic drawings.
We have 6 perception drawings.


## Save out data into a dataframe

In [ ]:
really_run_this = 1 
## compile data
if really_run_this:
    X = pd.DataFrame([session_id,age,trial_num,condition,category,submit_time,submit_date,num_strokes,svg,svg_times,png,draw_duration,avg_stroke_length,filename])
    X = X.transpose()
    X.columns = ['session_id','age','trial_num','condition','category','submit_time','submit_date','num_strokes','svg','svg_times','png','draw_duration', 'avg_stroke_length','filename']